# Behavioral Cloning-Project

In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import cv2

data_dir = os.path.join('D:', 'Simulation_Data', 'Complete')
log_csv = os.path.join(data_dir, 'driving_log.csv')

images_df = pd.read_csv(log_csv, 
                        names=['Center image', 'Left image', 'Right image', 'Steering', 'Throttle', 'Brake', 'Speed'], 
                        usecols=['Center image', 'Left image', 'Right image', 'Steering'])

print(images_df.loc[5])

Center image    D:\Simulation_Data\Complete\IMG\center_2018_10...
Left image      D:\Simulation_Data\Complete\IMG\left_2018_10_2...
Right image     D:\Simulation_Data\Complete\IMG\right_2018_10_...
Steering                                                -0.122066
Name: 5, dtype: object


In [2]:
from sklearn.model_selection import train_test_split
from random import shuffle

# Create training and validation dataset
train_samples, validation_samples = train_test_split(images_df, test_size=0.2)

def ImageBatchGenerator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_image_name, left_image_name, right_image_name, steering = batch_sample[0:4]
                
                center_image = cv2.imread(center_image_name)
                images.append(center_image)
                angles.append(steering)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)
            
train_generator = ImageBatchGenerator(train_samples.values)
validation_generator = ImageBatchGenerator(validation_samples.values)

## PilotNet model

In [ ]:
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten, Cropping2D, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

img_width = 320
img_height = 160

input = Input(shape=(img_height, img_width, 3))
x = Lambda(lambda x: x/255.0-0.5)(input)
x = Cropping2D(cropping=((70, 25), (0, 0)))(x)
x = Conv2D(24, (5, 5), strides=(2, 2), activation='relu')(x)
x = Conv2D(36, (5, 5), strides=(2, 2), activation='relu')(x)
x = Conv2D(48, (5, 5), strides=(2, 2), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Flatten()(x)
x = Dense(100)(x)
x = Dense(50)(x)
x = Dense(10)(x)
x = Dense(1)(x)

model = Model(input=input, output=x)
model.compile(loss='mse', optimizer='Nadam')

tensorboard = TensorBoard(log_dir='./logs/PilotNet', batch_size=32, write_images=True, histogram_freq=1)
# Save the model according to the conditions
checkpoint = ModelCheckpoint('PilotNet.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

model.fit_generator(train_generator, 
                    validation_data=validation_generator, 
                    epochs=10, 
                    callbacks=[tensorboard, checkpoint, early])

C:\Users\bjoer\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\bjoer\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


## Xception - transfer learning model

In [ ]:
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

img_width = 150
img_height = 150

model = Xception(weights='imagenet',
                 include_top=False,
                 input_shape=(img_height, img_width, 3))

# Freeze all layers of pre-trained model
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1)(x)  # Regression output layer

model_final = Model(input=model.input, output=x)
model_final.compile(loss='mse', optimizer='Nadam')

# Save the model according to the conditions  
checkpoint = ModelCheckpoint('Xception_steering.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_final.fit_generator(train_generator, 
                          samples_per_epoch=len(train_samples), 
                          validation_data=validation_generator,
                          nb_val_samples=len(validation_samples), 
                          nb_epoch=10, 
                          callbacks=[checkpoint, early])